In [87]:
clear()

In [88]:
#Set up
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LassoCV
from sklearn.linear_model import RidgeCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn import tree
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.ensemble import ExtraTreesClassifier

<font size=5 >**Task 1: Build a model to predict the class labels of the test samples in 'test_data.csv'. Save the prediction results in 'test_result1.csv', in which each row is a prediction.**</font> 

In [89]:
data = pd.read_csv( "train_data.csv")
data

,Customer_Age,Gender,Dependent,Education,Marital,Income,KCategory,onBook,Relationship,Inactive,...,TransAmt,TransCt,Ct_41,Utilization,Importance,MustHave,Group,Essential,DropLevel,Class
0,41,F,2,Unknown,Married,Unknown,Blue,21,2,3,...,1146.726149,127,0.628,0.063747,0.187124,91,4,9.321512,2,1
1,55,M,3,Post-Graduate,Unknown,$120K +,Blue,40,3,1,...,1143.720166,67,0.914,0.064382,0.866746,31,0,9.496893,1,1
2,53,F,2,Graduate,Married,$40K - $60K,Blue,42,3,5,...,1145.281665,89,0.679,0.057632,0.480002,75,4,7.120417,2,1
3,40,M,3,Uneducated,Married,$80K - $120K,Blue,34,4,3,...,1144.227271,111,0.850,0.065624,0.999413,30,2,9.103963,9,1
4,43,F,4,Unknown,Single,Less than $40K,Blue,33,3,3,...,1145.688723,69,0.568,0.063944,0.820799,19,2,1.620013,6,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7590,43,F,2,Post-Graduate,Married,Less than $40K,Blue,37,3,3,...,1145.776743,28,0.273,0.063384,0.755764,80,5,6.413782,5,2
7591,62,F,0,High School,Unknown,Less than $40K,Blue,51,4,2,...,1147.670872,36,0.161,0.060858,0.519155,12,5,1.945979,5,2
7592,32,M,1,High School,Married,$60K - $80K,Blue,24,4,2,...,1141.712294,38,0.520,0.065771,0.292057,4,3,9.489171,1,1
7593,45,M,4,Graduate,Married,$80K - $120K,Blue,36,5,3,...,1145.958050,39,0.696,0.063182,0.048110,40,4,8.377884,2,2


In [90]:
#Methods Change Gender, Education, Marital, Income, KCategory, into categorical data
def gender(x):
    if x== "F": return 0
    else: return 1

def  edu(x):
    if x=="Uneducated": return 0
    elif x=="High School": return 1
    elif x=="College": return 2
    elif x=="Graduate": return 3
    elif x=="Post-Graduate": return 4
    elif x=="Doctorate": return 5
    else: return -99

def marital(x):
    if x=="Single": return 0
    elif x=="Married": return 1
    else: return -99

def income(x):
    if x=="Less than $40k": return 0
    elif x=="$40K - $60K": return 1
    elif x=="$60K - $80K": return 2
    elif x=="$80K - $120K": return 3
    elif x=="$120K +": return 4
    else: return -99

def KCategory(x):
    if x=="Blue":return 0
    elif x=="Silver": return 1
    elif x=="Gold": return 2
    else: return 3
    
data['Gender']=data['Gender'].apply(gender)
data['Education']=data['Education'].apply(edu)
data['Marital']=data['Marital'].apply(marital)
data['Income']=data['Income'].apply(income)
data['KCategory']=data['KCategory'].apply(KCategory)

In [91]:
X = data.drop(['Class'], axis=1)
Y=data['Class']

#Normalize numerical Data
data['onBook']=(data['onBook'] - data['onBook'].min()) / (data['onBook'].max() - data['onBook'].min()) 
data['Relationship']=(data['Relationship'] - data['Relationship'].min()) / (data['Relationship'].max() - data['Relationship'].min()) 
data['Inactive']=(data['Inactive'] - data['Inactive'].min()) / (data['Inactive'].max() - data['Inactive'].min()) 
data['TransAmt']=(data['TransAmt'] - data['TransAmt'].min()) / (data['TransAmt'].max() - data['TransAmt'].min()) 
data['TransCt']=(data['TransCt'] - data['TransCt'].min()) / (data['TransCt'].max() - data['TransCt'].min())
data['Ct_41']=(data['Ct_41'] - data['Ct_41'].min()) / (data['Ct_41'].max() - data['Ct_41'].min()) 
data['Utilization']=(data['Utilization'] - data['Utilization'].min()) / (data['Utilization'].max() - data['Utilization'].min()) 
data['MustHave']=(data['MustHave'] - data['MustHave'].min()) / (data['MustHave'].max() - data['MustHave'].min()) 
data['Group']=(data['Group'] - data['Group'].min()) / (data['Group'].max() - data['Group'].min()) 
data['Essential']=(data['Essential'] - data['Essential'].min()) / (data['Essential'].max() - data['Essential'].min())
data['DropLevel']=(data['DropLevel'] - data['DropLevel'].min()) / (data['DropLevel'].max() - data['DropLevel'].min())

#generate dummy variables

X=pd.get_dummies(X, columns=['Gender','Education','Marital','Income','KCategory'], drop_first=True) # create dummy variables
print(X.columns)
# split data randomly
train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size=0.4, random_state = 213)

Index(['Customer_Age', 'Dependent', 'onBook', 'Relationship', 'Inactive',
       'ContactNum', 'CLimit', 'Revolving', 'Open', 'Amt_41', 'TransAmt',
       'TransCt', 'Ct_41', 'Utilization', 'Importance', 'MustHave', 'Group',
       'Essential', 'DropLevel', 'Gender_1', 'Education_0', 'Education_1',
       'Education_2', 'Education_3', 'Education_4', 'Education_5', 'Marital_0',
       'Marital_1', 'Income_1', 'Income_2', 'Income_3', 'Income_4',
       'KCategory_1', 'KCategory_2', 'KCategory_3'],
      dtype='object')


In [92]:
# train a linear regression
Linearreg = LinearRegression().fit(train_x, train_y)
print(Linearreg.intercept_, Linearreg.coef_, Linearreg.score(train_x, train_y))

#accuracy
lin_pred = Linearreg.predict(test_x)
linearAccuracy = 1-np.sum(np.abs(test_y-lin_pred))/len(test_y)
print(linearAccuracy)

-2.124871315451161 [ 9.59497512e-04  1.02414813e-02 -1.38701876e-03 -5.65965683e-02
  4.46231611e-02  4.27768773e-02  1.24552975e-04  1.40281303e-03
  1.18903749e-06 -6.70704917e-01  1.83021257e-03 -6.36105175e-03
 -3.12228572e-01  1.10085445e+00 -9.85106441e-03  2.76187587e-04
 -5.33874878e-04  2.63985188e-03 -6.28535815e-04 -7.38461918e-02
 -1.00452772e-02 -1.19463408e-02 -2.99170136e-02 -9.37283895e-03
 -1.38999864e-03 -1.88224428e-02  5.69990196e-03 -3.53185363e-02
 -7.29384217e-03  5.58608154e-03  1.00047965e-02  3.94558535e-03
  7.04320101e-03  3.41708722e-02  3.31228073e-02] 0.2962954277583859
0.7749385260501694


In [93]:
# train a logistic model
Logitreg = LogisticRegression(solver='newton-cg').fit(train_x, train_y)
print(Logitreg.intercept_, Logitreg.coef_, Logitreg.score(train_x, train_y))

#accuracy
pred = Logitreg.predict(test_x)
Logitregaccuracy=np.mean(pred == test_y)
print('Accuracy= ', Logitregaccuracy)
print(classification_report(test_y,pred))

[-0.01062014] [[ 1.06988970e-02  1.10734318e-01 -1.86880045e-02 -5.39687514e-01
   4.46712913e-01  4.33040847e-01 -7.58657128e-04  8.02444152e-03
   1.03330853e-05 -1.70949699e-02  6.06327807e-03 -6.49233248e-02
  -2.50648201e+00  1.17784342e-02 -1.18961521e-01  3.45502516e-03
  -2.62544858e-03  3.12758285e-02  2.09508209e-03 -7.48349796e-01
  -8.01256882e-02 -6.96651609e-03 -2.09139681e-01 -6.07967706e-02
  -1.08761272e-01 -1.76319691e-01  1.74197456e-02 -4.70463351e-01
  -1.19218321e-01  2.55634448e-02  4.91130597e-02  5.53843359e-03
   2.10723273e-02  2.50617686e-01  5.01287734e-02]] 0.8878648233486943
Accuracy=  0.8953258722843976
              precision    recall  f1-score   support

           1       0.91      0.97      0.94      2550
           2       0.78      0.48      0.60       488

    accuracy                           0.90      3038
   macro avg       0.85      0.73      0.77      3038
weighted avg       0.89      0.90      0.88      3038



/Users/liaohang/Library/Python/3.9/lib/python/site-packages/scipy/optimize/_linesearch.py:457: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/liaohang/Library/Python/3.9/lib/python/site-packages/scipy/optimize/_linesearch.py:306: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/liaohang/Library/Python/3.9/lib/python/site-packages/sklearn/utils/optimize.py:210: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  warnings.warn(


In [94]:
# train a tree
T = [1,2,3,4,5,6,7,8,9,10]
for t in T:
    aTreeClassifier = tree.DecisionTreeClassifier(max_depth=t)
    aTreeClassifier = aTreeClassifier.fit(train_x, train_y)
    #accuracy
    pred = aTreeClassifier.predict(test_x)
    Treeaccuracy1=np.mean(pred == test_y)
    print('Accuracy for ',t,' = ', Treeaccuracy1)
    

Accuracy for  1  =  0.8393680052666228
Accuracy for  2  =  0.8808426596445029
Accuracy for  3  =  0.8808426596445029
Accuracy for  4  =  0.8834759710335747
Accuracy for  5  =  0.9055299539170507
Accuracy for  6  =  0.901909150757077
Accuracy for  7  =  0.9032258064516129
Accuracy for  8  =  0.901909150757077
Accuracy for  9  =  0.9094799210006583
Accuracy for  10  =  0.9045424621461488


We can find the best tree model is when max_depth = 9

In [95]:
#Best Tree model
#Max_depth = 9
aTreeClassifier = tree.DecisionTreeClassifier(max_depth=9)
aTreeClassifier = aTreeClassifier.fit(train_x, train_y)
print(aTreeClassifier.score(train_x, train_y))
#accuracy
pred = aTreeClassifier.predict(test_x)
BestTree=np.mean(pred == test_y)
print('Accuracy for 9 = ', BestTree)
print(classification_report(test_y,pred))

0.9418477068246653
Accuracy for 9 =  0.9084924292297564
              precision    recall  f1-score   support

           1       0.93      0.96      0.95      2550
           2       0.76      0.63      0.69       488

    accuracy                           0.91      3038
   macro avg       0.85      0.80      0.82      3038
weighted avg       0.90      0.91      0.90      3038



In [96]:
# train a KNN

K = [1,2,3,4,5,6,7,8,9,10]
for k in K:
    KNN = KNeighborsClassifier(n_neighbors=k, metric = 'euclidean')
    KNN.fit(train_x, train_y)
    pred = KNN.predict(test_x)
    KNNaccuracy=np.mean(pred == test_y)
    print('K =', k, ', Accuracy =', KNNaccuracy)

K = 1 , Accuracy = 0.7975641869651087
K = 2 , Accuracy = 0.837063857801185
K = 3 , Accuracy = 0.8169848584595129
K = 4 , Accuracy = 0.8387096774193549
K = 5 , Accuracy = 0.8337722185648453
K = 6 , Accuracy = 0.8383805134957208
K = 7 , Accuracy = 0.8344305464121132
K = 8 , Accuracy = 0.8449637919684002
K = 9 , Accuracy = 0.8416721527320605
K = 10 , Accuracy = 0.8433179723502304


We can find the best KNN model is when neighbors = 8

In [97]:
#Best KNN model
#Final output KKN when k = 8
KNN = KNeighborsClassifier(n_neighbors=8, metric = 'euclidean')
KNN.fit(train_x, train_y)

pred = KNN.predict(test_x)
BestKNN=np.mean(pred == test_y)
print('K =', 8, ', Accuracy =', BestKNN)
print(classification_report(test_y,pred))

K = 8 , Accuracy = 0.8449637919684002
              precision    recall  f1-score   support

           1       0.85      0.99      0.91      2550
           2       0.60      0.11      0.18       488

    accuracy                           0.84      3038
   macro avg       0.72      0.55      0.55      3038
weighted avg       0.81      0.84      0.80      3038



<font size=3 >**The best model is tree with max depth of 9**</font> 

In [65]:
####PREDICT####
new_data = pd.read_csv( "test_data.csv")
new_data['Gender']=new_data['Gender'].apply(gender)
new_data['Education']=new_data['Education'].apply(edu)
new_data['Marital']=new_data['Marital'].apply(marital)
new_data['Income']=new_data['Income'].apply(income)
new_data['KCategory']=new_data['KCategory'].apply(KCategory)
X_new = new_data
X_new = pd.get_dummies(X_new, columns=['Gender','Education','Marital','Income','KCategory'], drop_first=True) # createdummy variables
X_new.columns

Index(['Customer_Age', 'Dependent', 'onBook', 'Relationship', 'Inactive',
       'ContactNum', 'CLimit', 'Revolving', 'Open', 'Amt_41', 'TransAmt',
       'TransCt', 'Ct_41', 'Utilization', 'Importance', 'MustHave', 'Group',
       'Essential', 'DropLevel', 'Gender_1', 'Education_0', 'Education_1',
       'Education_2', 'Education_3', 'Education_4', 'Education_5', 'Marital_0',
       'Marital_1', 'Income_1', 'Income_2', 'Income_3', 'Income_4',
       'KCategory_1', 'KCategory_2', 'KCategory_3'],
      dtype='object')

In [66]:
y_TreePredict=aTreeClassifier.predict(X_new)
np.savetxt('test_result1.csv', y_TreePredict, delimiter=',')

In [67]:
new=pd.read_csv( "test_result1.csv")
new.columns = ['Class']
new=new[new['Class'] == 2]
print(new)

      Class
1       2.0
8       2.0
15      2.0
20      2.0
21      2.0
...     ...
2496    2.0
2498    2.0
2507    2.0
2519    2.0
2523    2.0

[333 rows x 1 columns]


In [68]:
c1 = pd.read_csv( "test_result1.csv")
c1.columns = ['Class']
Y1 = c1['Class']

In [69]:
print(X.columns)
# Create the ExtraTreesClassifier model
model = ExtraTreesClassifier()

# Fit the model to the data
model.fit(X, Y)

importance_scores = {}
# Print feature importances
for feature, importance in zip(X, model.feature_importances_):
    importance_scores[feature] = importance
    
# Select top 4 features
top_features = sorted(importance_scores, key=importance_scores.get, reverse=True)[:4]

print("Top 4 features:")
for feature in top_features:
    print(f"{feature}: {importance_scores[feature]}")


Index(['Customer_Age', 'Dependent', 'onBook', 'Relationship', 'Inactive',
       'ContactNum', 'CLimit', 'Revolving', 'Open', 'Amt_41', 'TransAmt',
       'TransCt', 'Ct_41', 'Utilization', 'Importance', 'MustHave', 'Group',
       'Essential', 'DropLevel', 'Gender_1', 'Education_0', 'Education_1',
       'Education_2', 'Education_3', 'Education_4', 'Education_5', 'Marital_0',
       'Marital_1', 'Income_1', 'Income_2', 'Income_3', 'Income_4',
       'KCategory_1', 'KCategory_2', 'KCategory_3'],
      dtype='object')
Top 4 features:
TransCt: 0.16470279392475656
Ct_41: 0.08601637632177077
Relationship: 0.06517560492359588
ContactNum: 0.05584329767219209


max: TransCt,Ct_41,Relationship,ContactNum

<font size=5 >**Task 2: Build a prediction model that use at most 4 features. Save the prediction results of the test samples in 'test_result2.csv', in which each row is a prediction..**</font> 

In [70]:
clear()

In [71]:
#Set up
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LassoCV
from sklearn.linear_model import RidgeCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn import tree
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.ensemble import ExtraTreesClassifier

In [72]:
data = pd.read_csv( "train_data.csv")
X2 = data[['TransCt','Ct_41','Relationship','ContactNum']]
Y2 = data['Class']

# split data randomly
train_x2, test_x2, train_y2, test_y2 = train_test_split(X2, Y2, test_size=0.4, random_state = 213)

#Train Tree models
T = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
TreeAccuracy = [0] * len(T)  # initialize with zeros

for i, k in enumerate(T):
    TreeClassifier = tree.DecisionTreeClassifier(max_depth=k)
    TreeClassifier= TreeClassifier.fit(train_x2, train_y2)
    Tree_pred = TreeClassifier.predict( test_x2 )
    treeAccuracy= sum( Tree_pred == test_y2 ) / len( test_y2)
    TreeAccuracy[i] = treeAccuracy
    #print(f'With Depth of {k}, Tree Accuracy = {treeAccuracy}')

best_Tree=max(TreeAccuracy)
max_t = T[TreeAccuracy.index(best_Tree)]
print("Best Tree is Tree with depth=",max_t)

Best Tree is Tree with depth= 8


In [73]:
#Best Tree model
#Max_depth = 8
aTreeClassifier = tree.DecisionTreeClassifier(max_depth=max_t)
aTreeClassifier = aTreeClassifier.fit(train_x2, train_y2)
print(aTreeClassifier.score(train_x2, train_y2))
#accuracy
pred = aTreeClassifier.predict(test_x2)
BestTree=np.mean(pred == test_y2)
print('Accuracy for 8 = ', BestTree)
print(classification_report(test_y2,pred))

0.9111257406188282
Accuracy for 8 =  0.9025674786043449
              precision    recall  f1-score   support

           1       0.93      0.95      0.94      2550
           2       0.72      0.65      0.68       488

    accuracy                           0.90      3038
   macro avg       0.83      0.80      0.81      3038
weighted avg       0.90      0.90      0.90      3038



In [74]:
####PREDICT####
new_data = pd.read_csv( "test_data.csv")
X_new = new_data[['TransCt','Ct_41','Relationship','ContactNum']]
y_TreePredict=aTreeClassifier.predict(X_new)
np.savetxt('test_result2.csv', y_TreePredict, delimiter=',')

In [75]:
new=pd.read_csv( "test_result2.csv")
new.columns = ['Class']
new=new[new['Class'] == 2]
new

,Class
1,2.0
8,2.0
15,2.0
21,2.0
37,2.0
...,...
2498,2.0
2507,2.0
2519,2.0
2520,2.0
